<a href="https://colab.research.google.com/github/AndreRab/Polish-poems-generation/blob/main/polish_poems_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv('polskie_wiersze.csv')

In [ ]:
def build_semantic_rhyme_dict_fast(df, model, suffix_len=3, min_word_len=4):
    unique_words = set()
    for text in tqdm(df.text, desc="Extracting words"):
        words = re.findall(r'\b[a-ząćęłńóśźż]{' + str(min_word_len) + r',}\b', text.lower())
        unique_words.update(words)

    word_vectors = {}
    for word in tqdm(unique_words, desc="Caching vectors"):
        try:
            word_vectors[word] = model.get_word_vector(word)
        except Exception:
            continue

    suffix_groups = defaultdict(list)
    for word in word_vectors:
        suffix = word[-suffix_len:]
        suffix_groups[suffix].append(word)

    rhyme_dict = {}
    for word in tqdm(word_vectors, desc="Building rhyme dictionary"):
        suffix = word[-suffix_len:]
        rhymes = [w for w in suffix_groups[suffix] if w != word]

        similarities = [
            (w, cosine_similarity(word_vectors[word], word_vectors[w]))
            for w in rhymes if w in word_vectors
        ]
        rhymes_sorted = [w for w, _ in sorted(similarities, key=lambda x: x[1], reverse=True)]

        if rhymes_sorted:
            rhyme_dict[word] = rhymes_sorted

    return rhyme_dict

rhyme_dict = build_semantic_rhyme_dict_fast(data, model)

Building rhyme dictionary: 100%|██████████| 223459/223459 [40:56<00:00, 90.96it/s]


In [ ]:
import pickle

with open('my_dict.pkl', 'wb') as f:
    pickle.dump(rhyme_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving my_dict.pkl to my_dict (1).pkl


In [ ]:
import os
import pickle
file_path = 'my_dict.pkl'
with open(file_path, 'rb') as f:
    rhyme_dict = pickle.load(f)

In [ ]:
len(rhyme_dict)

221784

In [ ]:
import random
def get_random_word(rhyme_dict):
    return random.choice(list(rhyme_dict.keys()))

def get_rhyme(word, rhyme_dict):
    words = rhyme_dict.get(word, [])
    return words[random.randint(0, min(len(words) - 1, 5))]

def generate_line(prompt, model, tokenizer, max_tokens=12):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7,          # Less randomness for structure
        top_k=100,                # Still enough variety
        top_p=0.9,                # Sampling within a focused set
        repetition_penalty=1.2,   # Light repetition control
        pad_token_id=tokenizer.eos_token_id,
    )
    full_text = tokenizer.decode(output[0], skip_special_tokens=True)
    result = full_text[len(prompt):].strip().split('\n')[0].strip()
    return result

In [ ]:
def generate_rhymed_poem(num_couplets, model, tokenizer, rhyme_dict, system_prompt ='Napisz wiersz:\n'):
    prompt = system_prompt
    poem = ''

    for _ in range(num_couplets):
        last_word = get_random_word(rhyme_dict)
        prompt += f'[{last_word}] '

        line1 = generate_line(prompt, model, tokenizer)

        poem += line1 + "\n"
        prompt += line1 + "\n"

        rhyme = get_rhyme(last_word, rhyme_dict)

        prompt += f'[{rhyme}] '

        line2 = generate_line(prompt, model, tokenizer)
        poem += line2 + "\n"
        prompt += line2 + "\n"

    return {
        'prompt': prompt,
        'result': poem
    }

def generate_rhymed_poem_without_limit(model, tokenizer, rhyme_dict, system_prompt='Napisz wiersz:\n', max_couplets=10):
    prompt = system_prompt
    poem = ''
    couplet_count = 0

    while couplet_count < max_couplets:
        last_word = get_random_word(rhyme_dict)
        prompt += f'[{last_word}] '

        line1 = generate_line(prompt, model, tokenizer)
        if is_poem_end(line1):
            break

        poem += line1 + "\n"
        prompt += line1 + "\n"

        rhyme = get_rhyme(last_word, rhyme_dict)
        prompt += f'[{rhyme}] '

        line2 = generate_line(prompt, model, tokenizer)
        if is_poem_end(line2):
            break

        poem += line2 + "\n"
        prompt += line2 + "\n"

        couplet_count += 1

    return {
        'prompt': prompt,
        'result': poem
    }

def is_poem_end(line):
    return not line.strip() or '[KONIEC]' in line

In [ ]:
def clean_line(line):
    return re.sub(r'\s*\[\s*\]\s*', '', line)

def clean_word(text):
    text = re.sub(r'[\[\(\{\«\"\']+', '', text)
    text = re.sub(r'[\]\)\}\»\"\',:;.!?…\-—⁈]+', '', text)
    words = re.findall(r'\w{2,}', text)
    return words[-1] if words else ''

def add_first_word_to_line(text):
    res = []
    lines = text.split('\n')
    for line in lines:
        line = clean_line(line).strip()
        if not line:
            continue
        keyword = clean_word(line)
        if not keyword:
            continue
        extra_token = f'[{keyword}]'
        res.append(f'{extra_token} {line}')
    return '\n'.join(res)

In [ ]:
def prepare_poem_begin(poem_begin):
    lines = poem_begin.split('\n')
    res = ''
    for line in lines:
        res += (add_first_word_to_line(line) + '\n')
    return res

def generate_rhymed_poem_from_beginning(poem_start, num_couplets, model, tokenizer, rhyme_dict, system_prompt ="Napisz wiersz. Każdy wers powinien kończyć się podanym słowem [w nawiasie kwadratowym].\n"):
    prompt = system_prompt + prepare_poem_begin(poem_start)
    poem = poem_start

    for _ in range(num_couplets):
        last_word = get_random_word(rhyme_dict)
        prompt += f'[{last_word}] '

        line1 = generate_line(prompt, model, tokenizer)

        poem += line1 + "\n"
        prompt += line1 + "\n"

        rhyme = get_rhyme(last_word, rhyme_dict)

        prompt += f'[{rhyme}] '

        line2 = generate_line(prompt, model, tokenizer)
        poem += line2 + "\n"
        prompt += line2 + "\n"

    return {
        'prompt': prompt,
        'result': poem
    }

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
model_checkpoint = "AndreiRabau/poem-gpt-s"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/559k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.81M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
wiersze = []
for _ in tqdm(range(10)):
  wiersze.append(generate_rhymed_poem(4, model, tokenizer, rhyme_dict))

100%|██████████| 10/10 [01:40<00:00, 10.08s/it]


In [ ]:
for wiersz in wiersze:
  print(wiersz['result'])
  print('-------------')

tak mnie prędko zdysz,
łzy mi sypiesz.
to prośby prostej
otwartej — i pustej...
jak ten list do wichru, co w wet-
na wiatr utkwiła...
długo jeszcze czekam na twe modenie,
nie wiem, czy te kontynie,

-------------
w blasku słońca długi szereg
ty, i ja, coś mi mówił o dziwoląg
tyś mi wtedy mówiła, że się rozsunęła
wyszła z objęć twoich - lunął deszcz. -
widziałem dwoje umierających
dwunogich chłopców konających...
oto zmrok był - i oczy maszkinowe,
ręce twoje były szklane...

-------------
to jest, co w umyśle masz zamazanych.
nie wiesz nawet, że oczy przeświecałeś prześwietlonych
— słyszysz dyrektorów?
może z tych wąskich szkół i dróg i Ministrów
z tego związku, który panuje
z tego nadmiaru inteligencji panująca
tak, żeby było jeszcze jasnym światłem tworzącym
spełniania się ich wypełniania spełniającym

-------------
w grób go włożyli... On im pontę,
w dłoni trzymają fontanny.
Ówdzie się rozdzwoni metalicznym matematyczny
cyklop wielocyfrowy (ktoś fizyczny)
ja - jak tamtego przechodnia

In [ ]:
for _ in tqdm(range(50)):
  wiersze.append(generate_rhymed_poem(4, model, tokenizer, rhyme_dict))

for wiersz in wiersze[50:]:
  print(wiersz['prompt'])
  print('-------------')

100%|██████████| 50/50 [07:46<00:00,  9.32s/it]

Napisz wiersz:
[wisien] w oczach milczących jak echo wisien.
[pradawien] nagle jakby jakiś czarodziejski, pradawny
[jednaką] wielką siłą swą potężny i jednaką
[taką] światłością swoją dziwną i taką
[jasnom] tak samo, i tak podobne do jasnom -
[rosyanom] kwiaty te ukryte w ziemi rosachom.
[grubość] wtedy noc się robi coraz grubszą grubością
[długość] coraz głębsza jest księżyca wędrówka i długość;

-------------
Napisz wiersz:
[zaplątana] myśl w bezmiary zawita,
[wyplątana] tak jak wprzód wyplątana,
[wypłynąłeś] z tej strony wypłynąłeś.
[płynąłeś] już nie wypłyniesz — i płyniesz,
[gentlemani] wylądujesz znów gentlemani...
[hetmani] znów staniesz się hetmani!
[urodzoną] znów będziesz urożoną,
[zaręczoną] znowu zaręczoną....

-------------
Napisz wiersz:
[przewodniczyć] znowu będę przewodniczyć,
[wręczyć] znów się muszę tłumaczyć i wręczyć
[pragnieniem] tak samo jak i ja pragnieniem.
[chceniem] mieć będę gorzał swoje — nie chcę!
[centek] tyle to już mam za darmo centek
[dziewczątek] robót